In [54]:
import pyautogui
import pyperclip
import requests
import pandas as pd
import os
import re
import mysql.connector
import dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup

In [55]:
#Verifica se a pasta existe
if not os.path.exists("pages"):
    os.makedirs("pages")

In [56]:
#Conexão com o banco de dados
dotenv.load_dotenv()
host = os.getenv("DB_HOST_LOCAL")
user = os.getenv("DB_USERNAME_LOCAL")
database = os.getenv("DB_DATABASE_LOCAL")

connection = mysql.connector.connect(
    host = host,
    user = user,
    database = database,
)

cursor = connection.cursor()

In [ ]:
urls = [
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-paulo/',
    'https://www.zapimoveis.com.br/venda/apartamentos/sp+sao-paulo/',
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-paulo/',
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-paulo+centro/',
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-paulo+zona-norte/',
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-paulo+zona-oeste/',
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-paulo+zona-leste/',
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-paulo+zona-sul/',
    'https://www.zapimoveis.com.br/venda/casas/sp+carapicuiba++carapicuiba/',
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-caetano-do-sul/',
    'https://www.zapimoveis.com.br/venda/casas/sp+sao-bernardo-do-campo/',
    'https://www.zapimoveis.com.br/venda/casas/sp+santo-andre/',]

for index, url in enumerate(urls):
    #Abre o navegador
    chrome_icon = pyautogui.locateOnScreen('img\\chrome_icon.png', confidence=0.8)
    if chrome_icon:
        pyautogui.click(pyautogui.center(chrome_icon))
        print("Chrome clicado!")
    else:
        print("Ícone do Chrome não encontrado.")
    sleep(1.5)

    #Abre uma pagina anonima
    modo_visitante = pyautogui.locateOnScreen('img\\modo_visitante.png', confidence=0.8)
    if modo_visitante:
        print(modo_visitante)
        pyautogui.click(pyautogui.center(modo_visitante))
        print("Modo Visitante clicado!")
    else:
        print("Ícone do Chrome não encontrado.")

    sleep(0.7)

    #Acessa o html da pagina
    pyautogui.write(url, interval=0.1)
    pyautogui.press('enter')

    screen_width, screen_height = pyautogui.size()
    pyautogui.moveTo(screen_width / 2, screen_height / 2)

    for i in range(17):
        pyautogui.scroll(-2100)
        print(i)
        sleep(6)

    #Seleciona tudo e copia o html
    pyautogui.hotkey('ctrl', 'u')
    sleep(10)
    pyautogui.hotkey('ctrl', 'a')
    sleep(1)
    pyautogui.hotkey('ctrl', 'c')


    #Armazena o que foi copiado
    html = pyperclip.paste()


    #Escreve em um arquivo o html copiado
    with open(f"pages\\pagina_zapimoveis_page1.html", "w", encoding="utf-8") as f:
        f.write(html)

    pages = 99


    for i in range(1, pages + 1):
        sleep(5)
        link = pyautogui.locateOnScreen('img\\link.png', confidence=0.8)
        if link:
            pyautogui.click(pyautogui.center(link))
        if i <= 9:
            pyautogui.press('right')
            pyautogui.press('backspace')
            pyautogui.write(str(i + 1), interval=0.1)
        else:
            pyautogui.press('right')
            pyautogui.press('backspace', presses=2)
            pyautogui.write(str(i + 1), interval=0.1)
        pyautogui.press('enter')
        sleep(5)
        pyautogui.hotkey('ctrl', 'a')
        sleep(1)
        pyautogui.hotkey('ctrl', 'c')
        html = pyperclip.paste()
        with open(f"pages\\pagina_zapimoveis{index}_page{i + 1}.html", "w", encoding="utf-8") as f:
            f.write(html)

    pyautogui.hotkey('alt', 'f4')

Chrome clicado!
Box(left=np.int64(484), top=np.int64(829), width=184, height=49)
Modo Visitante clicado!
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [ ]:
df = pd.DataFrame()

all_links_extends = []
for i in range(len(os.listdir("pages"))):
    with open(f"pages\\pagina_zapimoveis_page{i + 1}.html", "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, 'html.parser')
        links_tag_a = [a['href'] for a in soup.find_all('a', href=True)
                if a['href'].startswith("https://www.zapimoveis.com.br/imovel/")]

        script_links = []
        for script in soup.find_all("script"):
            if script.string:
                found = re.findall(r"https://www\.zapimoveis\.com\.br/imovel/[^\s\"']+", script.string)
                script_links.extend(found)

        all_links = list(set(links_tag_a + script_links))
        all_links_extends.extend(all_links)
        

all_links_extends = list(set(all_links_extends))

links_limpos = [link.rstrip("\\/") for link in all_links_extends]

all_links_extends_limpo = [(link,) for link in links_limpos]

query = """
INSERT INTO amostras (link)
VALUES (%s)
ON DUPLICATE KEY UPDATE link = VALUES(link)
"""

print(len(all_links_extends_limpo))

cursor.executemany(query, all_links_extends_limpo)
connection.commit()

2778


In [61]:
query = """
SELECT link FROM amostras
"""

cursor.execute(query)
resultados = cursor.fetchall()

for link in resultados:
    print(link)
print(len(resultados))
# connection.commit()


# for url in all_links_extends:
#     try:
#         driver = webdriver.Chrome()
#         driver.get(url)
#         sleep(5)
        
#         elements = driver.find_elements(By.CLASS_NAME, "amenities-item-text")
#         preço = driver.find_elements(By.CLASS_NAME, "price-info-value")
#         endereço = driver.find_elements(By.CLASS_NAME, "address-info-value")
#         descricao = driver.find_elements(By.CLASS_NAME, "description__content--text")
#         property_data = {"Link": url}
        
#         for element in elements:
#             text = element.text.lower().strip()
            
#             if "m²" in text or "m2" in text or "metros quadrados" in text:
#                 property_data["Área"] = re.search(r'\d+', text).group() if re.search(r'\d+', text) else None
#             elif "quartos" in text or "quarto" in text:
#                 property_data["Quartos"] = re.search(r'\d+', text).group() if re.search(r'\d+', text) else None
#             elif "banheiros" in text or "banheiro" in text:
#                 property_data["Banheiros"] = re.search(r'\d+', text).group() if re.search(r'\d+', text) else None
#             elif "vagas" in text or "vaga" in text:
#                 property_data["Vagas"] = re.search(r'\d+', text).group() if re.search(r'\d+', text) else None
#             elif "suítes" in text or "suites" in text or "suíte" in text or "suíte" in text:
#                 property_data["Suítes"] = re.search(r'\d+', text).group() if re.search(r'\d+', text) else None
        
#         property_data["Valor"] = preço[0].text if preço else None
#         property_data["Endereço"] = endereço[0].text if endereço else None
#         property_data["Descrição"] = descricao[0].text if descricao else None
        
#         df = pd.concat([df, pd.DataFrame([property_data])], ignore_index=True)
#     except Exception as e:
#         print(f"Error processing {url}: {str(e)}")
#     finally:
#         driver.quit()
#         df.head(10)

('https://www.zapimoveis.com.br/imovel/venda-2-quartos-com-piscina-parque-laguna-ii-formosa-110000m2-id-2797397313/',)
('https://www.zapimoveis.com.br/imovel/venda-2-quartos-com-piscina-parque-laguna-ii-formosa-110000m2-id-2797397313/\\',)
('https://www.zapimoveis.com.br/imovel/venda-3-quartos-com-churrasqueira-setor-faicalville-goiania-107m2-id-2794090045/',)
('https://www.zapimoveis.com.br/imovel/venda-3-quartos-com-churrasqueira-setor-faicalville-goiania-107m2-id-2794090045/\\',)
('https://www.zapimoveis.com.br/imovel/venda-3-quartos-com-piscina-vila-roque-sao-paulo-55m2-id-2739495290/',)
('https://www.zapimoveis.com.br/imovel/venda-3-quartos-com-piscina-vila-roque-sao-paulo-55m2-id-2739495290/\\',)
('https://www.zapimoveis.com.br/imovel/venda-apartamento-1-quarto-boa-viagem-recife-25m2-id-2797646461/',)
('https://www.zapimoveis.com.br/imovel/venda-apartamento-1-quarto-boa-viagem-recife-25m2-id-2797646461/\\',)
('https://www.zapimoveis.com.br/imovel/venda-apartamento-1-quarto-boa-vi

In [59]:
df.to_excel("zapimoveis.xlsx", index=False)